In [15]:
using PowerModels
using Ipopt

network_data = PowerModels.parse_file("CaseData/case5.m")

result = run_opf(network_data, DCPPowerModel, IpoptSolver())

println(result["solution"])
println(loss_ac =  Dict(name => data["pt"]+data["pf"] for (name, data) in result["solution"]["branch"]))

[warn | PowerModels]: this code only supports angmin values in -90 deg. to 90 deg., tightening the value on branch 4 from -360.0 to -60.0 deg.
[warn | PowerModels]: this code only supports angmax values in -90 deg. to 90 deg., tightening the value on branch 4 from 360.0 to 60.0 deg.
[warn | PowerModels]: this code only supports angmin values in -90 deg. to 90 deg., tightening the value on branch 1 from -360.0 to -60.0 deg.
[warn | PowerModels]: this code only supports angmax values in -90 deg. to 90 deg., tightening the value on branch 1 from 360.0 to 60.0 deg.
[warn | PowerModels]: this code only supports angmin values in -90 deg. to 90 deg., tightening the value on branch 5 from -360.0 to -60.0 deg.
[warn | PowerModels]: this code only supports angmax values in -90 deg. to 90 deg., tightening the value on branch 5 from 360.0 to 60.0 deg.
[warn | PowerModels]: this code only supports angmin values in -90 deg. to 90 deg., tightening the value on branch 2 from -360.0 to -60.0 deg.
[warn

KeyError: KeyError: key "branch" not found

In [40]:
#PowerModels.print_summary(result["solution"])
PowerModels.print_summary(network_data)
#PowerModels.component_table(network_data, "branch",["f_bus","t_bus"])

Metadata
  baseMVA: 100
  name: case5
  per_unit: true
  source_type: matpower
  source_version: 2.0.0

Table Counts
  bus: 5
  load: 3
  shunt: 0
  gen: 5
  storage: 0
  branch: 6
  dcline: 0


Table: bus
         bus_i, bus_type
      1:     1,        2
      2:     2,        1
      3:     3,        2
      4:     4,        3
      5:     5,        2

  default values:
    base_kv: 230.000
    vm: 1.000
    va: 0.000
    vmin: 0.900
    vmax: 1.100
    area: 1
    zone: 1


Table: load
         load_bus,    pd,    qd
      1:        2, 3.000, 0.986
      2:        3, 3.000, 0.986
      3:        4, 4.000, 1.315

  default values:
    status: 1


Table: gen
         gen_bus,    pg,  pmax,   qmin,  qmax
      1:       1, 0.400, 0.400, -0.300, 0.300
      2:       1, 1.700, 1.700, -1.275, 1.275
      3:       3, 3.235, 5.200, -3.900, 3.900
      4:       4, 0.000, 2.000, -1.500, 1.500
      5:       5, 4.665, 6.000, -4.500, 4.500

  default values:
    qg: 0.000
    vg: 1.000
    mbase

In [85]:
#### AC Optimal Power Flow ####

# This file provides a pedagogical example of modeling the AC Optimal Power
# Flow problem using the Julia Mathematical Programming package (JuMP) and the
# PowerModels package for data parsing.

# This file can be run by calling `include("ac-opf.jl")` from the Julia REPL or
# by calling `julia ac-opf.jl` in Julia v0.6.

# Developed by Line Roald (@lroald) and Carleton Coffrin (@ccoffrin)


###############################################################################
# 0. Initialization
###############################################################################

# Load Julia Packages
#--------------------
using PowerModels
using Ipopt
using JuMP


# Instancate a Solver
#--------------------

nlp_solver = IpoptSolver(print_level=0)
# note: print_level changes the amount of solver information printed to the terminal


# Load System Data
# ----------------

if VERSION < v"0.7.0"
    powermodels_path = Pkg.dir("PowerModels")
else
    powermodels_path = joinpath(dirname(pathof(PowerModels)), "..")
end

file_name = "$(powermodels_path)/test/data/matpower/case5.m"
# note: change this string to modify the network data that will be loaded

# load the data file
data = PowerModels.parse_file(file_name)

# Add zeros to turn linear objective functions into quadratic ones
# so that additional parameter checks are not required
PowerModels.standardize_cost_terms(data, order=2)

# use build_ref to filter out inactive components
ref = PowerModels.build_ref(data)[:nw][0]
# note: ref contains all the relevant system parameters needed to build the OPF model
# When we introduce constraints and variable bounds below, we use the parameters in ref.


###############################################################################
# 1. Building the Optimal Power Flow Model
###############################################################################

# Initialize a JuMP Optimization Model
#-------------------------------------
model = Model(solver = nlp_solver)


# Add Optimization and State Variables
# ------------------------------------

# Add power generation variables for each generator, including bounds
@variable(model, ref[:gen][i]["pmin"] <= pg[i in keys(ref[:gen])] <= ref[:gen][i]["pmax"])
# Add power flow variables for branches
@variable(model, -ref[:branch][l]["rate_a"] <= p[(l,i,j) in ref[:arcs]] <= ref[:branch][l]["rate_a"])

# Add Objective Function
# ----------------------

# Minimize cost power generation
# assumes costs are given as quadratic functions
@objective(model, Min,
    sum(gen["cost"][1]*pg[i]^2 + gen["cost"][2]*pg[i] + gen["cost"][3] for (i,gen) in ref[:gen])
)

# Add Constraints
# ---------------

# Nodal power balance constraints
for (i,bus) in ref[:bus]
    # Build a list of the loads and shunt elements connected to the bus i
    bus_loads = [ref[:load][l] for l in ref[:bus_loads][i]]
    bus_shunts = [ref[:shunt][s] for s in ref[:bus_shunts][i]]

    # Power balance
    @constraint(model,
        sum(p[a] for a in ref[:bus_arcs][i]) == # sum of power flow on lines from bus i
        sum(pg[g] for g in ref[:bus_gens][i]) - # sum of power generation at bus i
        sum(load["pd"] for load in bus_loads)   # sum of active load consumption at bus i
    )
end

# Branch flow limits
for (i, branch) in ref[:branch]
    # build the from variable id of the i-th branch
    f_idx = (i, branch["f_bus"], branch["t_bus"])
    # build the to variable id of the i-th branch
    t_idx = (i, branch["t_bus"], branch["f_bus"])
     # note: it is necessary to distinguish between the from and to sides of a branch due to power losses

     p_fr = p[f_idx]
     p_to = p[t_idx]
     # note: p_fr, p_to are references to the optimization variable p[index]

     @constraint( model, p_fr + p_to == 0)

end

###############################################################################
# 3. Solve the Optimal Power Flow Model and Review the Results
###############################################################################

# Solve the optimization problem
status = solve(model)

# Check the value of the objective function
cost = getobjectivevalue(model)
println("The cost of generation is $(cost).")

# Check the value of an optimization variable
# Example: Active power generated at generator 1
pg1 = getvalue(pg[1])
println("The active power generated at generator 1 is $(pg1*ref[:baseMVA]) MW.")
# note: the optimization model is in per unit, so the baseMVA value is used to restore the physical units


[warn | PowerModels]: reversing the orientation of branch 6 (4, 3) to be consistent with other parallel branches
[warn | PowerModels]: the voltage setpoint on generator 4 does not match the value at bus 4
[warn | PowerModels]: the voltage setpoint on generator 1 does not match the value at bus 1
[warn | PowerModels]: the voltage setpoint on generator 5 does not match the value at bus 10
[warn | PowerModels]: the voltage setpoint on generator 2 does not match the value at bus 1
[warn | PowerModels]: the voltage setpoint on generator 3 does not match the value at bus 3
[info | PowerModels]: removing 1 cost terms from generator 4: [4000.0, 0.0]
[info | PowerModels]: removing 1 cost terms from generator 1: [1400.0, 0.0]
[info | PowerModels]: removing 1 cost terms from generator 5: [1000.0, 0.0]
[info | PowerModels]: removing 1 cost terms from generator 2: [1500.0, 0.0]
[info | PowerModels]: removing 1 cost terms from generator 3: [3000.0, 0.0]
[warn | PowerModels]: Updated generator 4 cost

In [37]:
model

Minimization problem with:
 * 12 linear constraints
 * 19 variables
Solver is Ipopt

In [86]:
pg_1 = getvalue(pg[:])
println("The active power generated at generators is $(pg_1)")

p_1 = getvalue(p[:])
println("The active power generated at along each branch is $(p_1)")

for (l,i,j) in ref[:arcs]
    
    println(l, " ", i, " ", j, "    ",getvalue(p[(l,i,j)]))
    
end

The active power generated at generators is [0.0, 1.7, 1.9, 6.0, 0.4]
The active power generated at along each branch is [-2.13022, -0.0689956, 3.03877, -3.86978, -0.584498, -0.584498, 2.931, 2.13022, 0.0689956, -3.03877, 3.86978, 0.584498, 0.584498, -2.931]
7 4 10    -2.130222114165274
4 2 3    -0.06899556471133735
2 1 4    3.0387735373664837
3 1 10    -3.8697779457846164
5 3 4    -0.5844978208159999
6 3 4    -0.5844978208159999
1 1 2    2.9310044352886626
7 10 4    2.130222114165274
4 3 2    0.06899556471133735
2 4 1    -3.0387735373664837
3 10 1    3.8697779457846164
5 4 3    0.5844978208159999
6 4 3    0.5844978208159999
1 2 1    -2.9310044352886626


In [135]:
#--------------------
using PowerModels
using Ipopt
using JuMP

function build_mn_data(base_data; replicates::Int=2)
    mp_data = PowerModels.parse_file(base_data)
    return PowerModels.replicate(mp_data, replicates)
end

# Instancate a Solver
#--------------------

nlp_solver = IpoptSolver(print_level=0)
# note: print_level changes the amount of solver information printed to the terminal


# Load System Data
# ----------------

if VERSION < v"0.7.0"
    powermodels_path = Pkg.dir("PowerModels")
else
    powermodels_path = joinpath(dirname(pathof(PowerModels)), "..")
end

file_name = "ModelData/case5"
file_ext = ".m"
periods = 6
# note: change this string to modify the network data that will be loaded

# load network data
data = build_mn_data(string(file_name,"_",1,file_ext),replicates = periods)



# Add zeros to turn linear objective functions into quadratic ones
# so that additional parameter checks are not required
PowerModels.standardize_cost_terms(data, order=2)

# use build_ref to filter out inactive components
#ref = PowerModels.build_ref(data)[:nw][0]
ref = PowerModels.build_ref(data)
# note: ref contains all the relevant system parameters needed to build the OPF model
# When we introduce constraints and variable bounds below, we use the parameters in ref.


[warn | PowerModels]: this code only supports angmin values in -90 deg. to 90 deg., tightening the value on branch 4 from -360.0 to -60.0 deg.
[warn | PowerModels]: this code only supports angmax values in -90 deg. to 90 deg., tightening the value on branch 4 from 360.0 to 60.0 deg.
[warn | PowerModels]: this code only supports angmin values in -90 deg. to 90 deg., tightening the value on branch 1 from -360.0 to -60.0 deg.
[warn | PowerModels]: this code only supports angmax values in -90 deg. to 90 deg., tightening the value on branch 1 from 360.0 to 60.0 deg.
[warn | PowerModels]: this code only supports angmin values in -90 deg. to 90 deg., tightening the value on branch 5 from -360.0 to -60.0 deg.
[warn | PowerModels]: this code only supports angmax values in -90 deg. to 90 deg., tightening the value on branch 5 from 360.0 to 60.0 deg.
[warn | PowerModels]: this code only supports angmin values in -90 deg. to 90 deg., tightening the value on branch 2 from -360.0 to -60.0 deg.
[warn

Dict{Symbol,Any} with 1 entry:
  :nw => Dict{Int64,Any}(4=>Dict{Symbol,Any}(:arcs_to=>Tuple{Int64,Int64,Int64}…

In [110]:
#PowerModels.print_summary(data)
data2 = data
data2["load"]

Dict{String,Any} with 3 entries:
  "1" => Dict{String,Any}("load_bus"=>2,"status"=>1,"qd"=>0.9861,"pd"=>3.0,"ind…
  "2" => Dict{String,Any}("load_bus"=>3,"status"=>1,"qd"=>0.9861,"pd"=>3.0,"ind…
  "3" => Dict{String,Any}("load_bus"=>4,"status"=>1,"qd"=>1.3147,"pd"=>4.0,"ind…

In [150]:
#PowerModels.print_summary(data["nw"][1])
data
data["nw"][string(periods)]["load"]["2"]

Dict{String,Any} with 5 entries:
  "load_bus" => 3
  "status"   => 1
  "qd"       => 0.9861
  "pd"       => 3.0
  "index"    => 2